In [12]:
"""
Challenge #2
영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.


Hint.
LLM이 답변 형식을 학습하도록 다양한 영화에 대한 예시를 만들어야 합니다.
예시는 과제의 요구조건을 만족시키려면 감독, 주요 출연진, 예산, 흥행 수익, 장르, 간략한 줄거리 가 포함되어야 합니다. LLM이 답변 형식을 효과적으로 학습하려면 모든 예시는 동일한 형식을 유지해야 합니다.
"""
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature= 0.1,
    streaming=False,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [9]:
#예시 데이터 구성
examples = [
    {
        "input": "기생충",
        "output": """영화 제목: 기생충
감독: 봉준호
주요 출연진: 송강호, 이선균, 조여정, 최우식, 박소담
예산: 약 135억 원
흥행 수익: 약 $263 million
장르: Drama, Thriller
간략한 줄거리: 가난한 가족이 부잣집에 취업하며 벌어지는 계층 간의 갈등과 비극을 그린 영화입니다."""
    },
    {
        "input": "The Dark Knight",
        "output": """영화 제목: The Dark Knight
감독: Christopher Nolan
주요 출연진: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine
예산: $185 million
흥행 수익: $1.005 billion
장르: Action, Crime, Drama
간략한 줄거리: 배트맨이 고담시를 위협하는 조커와 맞서며 정의와 희생의 의미를 탐구하는 이야기입니다."""
    },
    {
        "input": "세 얼간이",
        "output": """영화 제목: 세 얼간이
감독: 라지쿠마르 히라니
주요 출연진: 아미르 칸, 카리나 카푸르, 마드하반, 샤르만 조시
예산: 약 ₹550 million (약 $7 million)
흥행 수익: 약 ₹4.6 billion (약 $90 million)
장르: Comedy, Drama
간략한 줄거리: 세 친구가 엄격한 공과대학에서 학문, 우정, 사랑을 통해 삶의 진정한 의미를 찾아가는 유쾌하고 감동적인 이야기입니다."""
    },
]

In [11]:
#FewShotPromptTemplate 생성
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="""영화 제목: {input}
{output}"""
)


few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="영화 제목을 입력하면 해당 영화에 대한 정보를 제공해드립니다. 다음 형식을 유지하세요:\n",
    suffix="영화 제목: {input}",
    input_variables=["input"]
)

In [14]:
# LLMChain 생성
movie_chain = LLMChain(llm=llm, prompt=few_shot_prompt)

# 영화 정보 생성
def get_movie_info(movie_title):
    result = movie_chain.run(movie_title)
    return result

# 예시 실행
movie_title = "7번방의 선물"
movie_info = get_movie_info(movie_title)

#
print(movie_info)

영화 제목: 7번방의 선물
감독: 이환경
주요 출연진: 류승룡, 박신혜, 오달수, 박원상
예산: 약 50억 원
흥행 수익: 약 $91 million
장르: Drama, Comedy
간략한 줄거리: 잘못된 혐의로 가혹한 형벌을 받게 된 소년들이 함께하는 감동적인 우정과 성장을 그린 영화입니다.
